In [29]:
from lark import Lark, Transformer

mu_parser = Lark.open("grammars/mu.lark", parser="earley")
formula = mu_parser.parse(open("../examples/test.mu").read())
print(formula.pretty())

start
  formula
    box
      a
      formula	true



In [30]:
class TreeToJson(Transformer):
    def start(self, f):
        (f,) = f
        return f
    def formula(self, f):
        (f,) = f
        return f
    
    TRUE = lambda self, _: {"val": True}
    FALSE = lambda self, _: {"val": False}
    def VAR(self, f):
        (f,) = f
        return {"var": f}
    def AL(self, f):
        (f,) = f
        return f
    def land(self, f):
        return {"and": f}
    def lor(self, f):
        return {"or": f}
    def diamond(self, f):
        (a, f) = f
        #(a,) = a
        return {"diamond": [a, f]}
    def box(self, f):
        #print(f)
        (a, f) = f
        #(a,) = a
        return {"box": [a, f]}
    def mu(self, f):
        return {"mu": f}
    def nu(self, f):
        return {"nu": f}

print(TreeToJson().transform(formula))

{'box': ['a', {'val': True}]}


In [7]:
def reduceFormula(t):
    operand = list(t.keys())[0]
    arguments = t[operand]

    def negate(t, x):
        o = list(t.keys())[0]
        a = t[o]

        if o == "var":
            if a == x["var"]:
                return {"neg": t}
            else:
                return t
        elif o == "val":
            return t
        elif o == "and" or o == "or":
            return {o: [negate(arg, x) for arg in a]}
        elif o == "box" or o == "diamond":
            return {o: [a[0], negate(a[1], x)]}
        elif o == "mu" or o == "nu":
            v = arguments[0]
            f = arguments[1]
            return {o: [v, negate(f, x)]}
        else:
            return {o: negate(a, x)}

    if operand == "val":
        if arguments == False:
            return {"neg": {"val": True}}
        else:
            return t
    elif operand == "var":
        return t
    elif operand == "diamond":
        return {"neg": {"box": [arguments[0], {"neg": reduceFormula(arguments[1])}]}}
    elif operand == "box":
        return {"box": [arguments[0], reduceFormula(arguments[1])]}
    elif operand == "and" or operand == "or":
        return {operand: [reduceFormula(arg) for arg in arguments]}
    elif operand == "mu":
        v = arguments[0]
        f = arguments[1]
        return {"neg": {"nu": [v, reduceFormula(negate(f, v))]}}
    elif operand == "nu":
        v = arguments[0]
        f = arguments[1]
        return {operand: [v, reduceFormula(f)]}
    else:
        return {operand: reduceFormula(arguments)}

formula = reduceFormula(TreeToJson().transform(formula))
print(formula)


{'neg': {'nu': [{'var': 'X'}, {'or': [{'neg': {'box': ['a', {'neg': {'val': True}}]}}, {'box': ['b', {'and': [{'neg': {'val': True}}, {'neg': {'var': 'X'}}]}]}]}]}}


In [9]:
%store -r lts
def simple(lts, formula):
    states = lts.states
    operand = list(formula.keys())[0]
    arguments = formula[operand]
    if operand == "neg":
        return states - (simple(lts,arguments))
    if operand == "or":
        return simple(lts,arguments[0]) | simple(lts,arguments[1])
    if operand == "and":
        return simple(lts,arguments[0]) & simple(lts,arguments[1])
    if operand == "box":
        return lts.box(simple(lts,arguments[1]),arguments[0])
    if operand == "val":
        return states
    if operand == "var":
        return variables[arguments] 
    elif operand == "nu":
        variable = arguments[0]["var"]
        #we start with all the states
        variables[variable] = states
        newSol = simple(lts,arguments[0])
        while newSol != variables[variable]:
            newSol = simple(lts,arguments[0])
        return newSol
    else:
        return states

#use a dictoinary to keep track of the variables
variables = dict()  
simple(lts,formula)




set()